In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import torch
import torchvision
import pytorch_lightning as pl

import os
import PIL.Image as Image
import base64
import io
import json
import requests
from urllib.parse import urlencode
from k12libs.utils.nb_easy import k12ai_get_top_dir, RACEURL

np.__version__, torch.__version__, torchvision.__version__, pl.__version__

('1.19.4', '1.8.0.dev20210103+cu101', '0.9.0.dev20210103+cu101', '1.2.0')

## Dataset 

### Garbage

In [220]:
model = 'Resnet18'
root_dir = '/raceai/data/tmp/pl_rgarbage_resnet18'
data_root = '/raceai/data/datasets/rgarbage/'
input_size = 224
mean = [
    0.6535,
    0.6132,
    0.5643
]
std = [
    0.2165,
    0.2244,
    0.2416
]
num_classes = 4
data_source = f'{data_root}/train.json'
valid_data_source = f"{data_root}/val.json"
test_data_source = f"{data_root}/test.json"
resume_weights = f"/raceai/data/ckpts/rgarbage/pl_resnet18_acc85.pth"

### Mnist

In [17]:
model = 'Resnet18'
root_dir = '/raceai/data/tmp/pl_rmnist_resnet18'
data_root = '/raceai/data/datasets/rmnist/'
input_size = 28
mean = [
    0.1362,
    0.1362,
    0.1362
]
std = [
    0.2893,
    0.2893,
    0.2893
]
num_classes = 10
data_source = f'{data_root}/train.json'
valid_data_source = f"{data_root}/val.json"
test_data_source = f"{data_root}/test_rotation.json"
resume_weights = f"/raceai/data/ckpts/rmnist/pl_resnet18_rotation.pth"
# resume_weights = f"/raceai/data/ckpts/rmnist/pl_resnet18_normal.pth"

### CleanerRobot 

In [50]:
model = 'Resnet18'
root_dir = '/raceai/data/tmp/pl_cleaner_robot_resnet18'
data_root = '/raceai/data/datasets/cleaner_robot/'
input_size = 224
mean = [
    0.7566,
    0.7323,
    0.7016
]
std = [
    0.1805,
    0.194,
    0.2195
]
num_classes = 3
data_source = f'{data_root}/train.json'
valid_data_source = f"{data_root}/val.json"
test_data_source = f"{data_root}/test.json"
resume_weights = f"/raceai/data/ckpts/cleaner_robot/pl_resnet18_acc90.pth"

## RestAPI

### Training

In [26]:
optimizer = {
    "class_name": "torch.optim.Adam",
    "params": {
        "lr": 0.001,
        "betas": [0.9, 0.999],
        "weight_decay": 0,
        "amsgrad": True
    }
}

# optimizer = {
#     "class_name": "torch.optim.SGD",
#     "params": {
#         "lr": 0.001,
#         "momentum": 0.9,
#         "weight_decay": 0,
#         "nesterov": True
#     }
# }
# 
# scheduler = {
#     "class_name": "torch.optim.lr_scheduler.StepLR",
#     "params": {
#         "step_size": 20,
#         "gamma": 0.1
#     }
# }

# scheduler = {
#     "class_name": "torch.optim.lr_scheduler.MultiStepLR",
#     "params": {
#         "milestones": [20, 40],
#         "gamma": 0.1
#     }
# }

scheduler = {
    "class_name": "torch.optim.lr_scheduler.ReduceLROnPlateau",
    "params": {
        "factor": 0.1,
        "patience": 3,
        "min_lr": 1e-6,
        "threshold": 1e-4,
        "verbose": True,
    }
}

In [67]:
early_stopping = {
    "class_name": "pytorch_lightning.callbacks.EarlyStopping",
    "params": {
        "monitor": "val_loss",
        "mode": "min",
        "patience": 6,
        "verbose": True,
        "min_delta": 0.0001
    }
}

model_checkpoint = {
    "class_name": "pytorch_lightning.callbacks.ModelCheckpoint",
    "params": {
        "monitor": "val_acc",
        "mode": "max",
        "save_top_k": 1,
        "period": 2,
        "save_weights_only": False,
        "filename": "best"
    }
}

logger = {
    "class_name": "pytorch_lightning.loggers.CSVLogger",
    "params": {
        "save_dir": root_dir,
        "name": "logs"
    }
}

random_rotation = {
    "class_name": "torchvision.transforms.RandomRotation",
    "params": {
        "degrees": 75
    }
}

reqdata = {
    "task": "cls.training.pl",
    "cfg": {
        "data": {
            "train": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": data_source,
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": mean,
                             "std": std,
                             "imgaugs": [random_rotation]
                         }
                    },
                    "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                        "drop_last": True,
                        "shuffle": True
                    }
                }
            },
            "valid": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": valid_data_source,
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": mean,
                             "std": std,
                         }
                     },
                     "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                        "drop_last": False
                    }
                 }
            },
            "test": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": test_data_source,
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": mean,
                             "std": std,
                         }
                     },
                    "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                    }
                 }
            }
        },
        "model": {
            "class_name": f"raceai.models.backbone.{model}",  
            "params": {
                "device": 'gpu',
                "num_classes": num_classes,
                "weights": True,
            }
        },
        "solver": {
            "optimizer": optimizer,
            "scheduler": scheduler,
            "trainer": {
                "class_name": "raceai.runner.pl.PlTrainer",
                "params": {
                    "callbacks": [model_checkpoint],
                    "logger": False,
                    "general": {
                        "max_epochs": 30,
                        "gpus": 1,
                        "resume_from_checkpoint": None,
                        "default_root_dir": root_dir
                    }
                }
            }
        }
    }
}

uri = f'{RACEURL}/raceai/framework/training'

resdata = json.loads(requests.post(url=uri, json=reqdata).text)
if 0 != resdata['errno']:
    if 'traceback' in resdata['result']:
        print(resdata['result']['traceback'])
    else:
        print(resdata)
else:
    print(resdata)

{'errno': 0, 'result': [{'test_acc': 0.9411764740943909}]}


### Inference(json)

In [77]:
reqdata = {
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.LocalDataLoader",  
            "params": {
                "data_source": test_data_source,
                "dataset": {
                    "class_name": "raceai.data.JsonFileDataset",
                     "params": {
                         "input_size": input_size,
                         "mean": mean,
                         "std": std
                     }
                 },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
             }
        },
        "model": {
            "class_name": f"raceai.models.backbone.{model}",  
            "params": {
                "device": 'gpu',
                "num_classes": num_classes,
                "weights": False
            }
        },
        "trainer": {
            "default_root_dir": root_dir,
            "gpus": 1,
            "resume_from_checkpoint": resume_weights
        }
    }
}

uri = f'{RACEURL}/raceai/framework/inference'

resdata = json.loads(requests.post(url=uri, json=reqdata).text)
if 0 != resdata['errno']:
    if 'traceback' in resdata['result']:
        print(resdata['result']['traceback'])
    else:
        print(resdata)
else:
    print(resdata)

{'errno': 0, 'result': [{'image_id': 2, 'image_path': '/raceai/data/datasets/cleaner_robot/imgs/other/paper_11.jpg', 'probs': [2.40847330132965e-05, 0.0015751824248582125, 0.9984007477760315], 'probs_sorted': {'indices': [2, 1, 0], 'values': [0.9984007477760315, 0.0015751824248582125, 2.40847330132965e-05]}}, {'image_id': 2, 'image_path': '/raceai/data/datasets/cleaner_robot/imgs/other/paper_15.jpg', 'probs': [0.31497329473495483, 0.01957719773054123, 0.6654495000839233], 'probs_sorted': {'indices': [2, 0, 1], 'values': [0.6654495000839233, 0.31497329473495483, 0.01957719773054123]}}, {'image_id': 2, 'image_path': '/raceai/data/datasets/cleaner_robot/imgs/other/cigarette_12.jpeg', 'probs': [4.701891612057807e-06, 7.6005103437637445e-06, 0.9999877214431763], 'probs_sorted': {'indices': [2, 1, 0], 'values': [0.9999877214431763, 7.6005103437637445e-06, 4.701891612057807e-06]}}, {'image_id': 0, 'image_path': '/raceai/data/datasets/cleaner_robot/imgs/ring/ring_4.jpg', 'probs': [0.9982069730

### Inference(url)

In [78]:
reqdata = {
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",  
            "params": {
                "data_source": [
                    "https://img14.360buyimg.com/n0/jfs/t1/115543/8/11330/109373/5efc3fcdE2dfb9242/39c1e05e0371cde9.jpg",
                    "https://jmy-pic.wejianzhan.com/0/pic/e621772e901d6d2f0ff072582e430173.jpg@f_webp,q_90",
                    "http://5b0988e595225.cdn.sohucs.com/images/20190201/2976462bd67042d5b1b14c1d0f4a6717.jpeg",
                    "/raceai/data/datasets/cleaner_robot/imgs/ring/ring_5.jpg",
                    "/raceai/data/datasets/cleaner_robot/imgs/cable/cable_5.jpg",
                ],
                "dataset": {
                    "class_name": "raceai.data.PredictListImageDataset",
                     "params": {
                         "input_size": input_size,
                         "mean": mean,
                         "std": std
                     }
                 },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
             }
        },
        "model": {
            "class_name": f"raceai.models.backbone.{model}",  
            "params": {
                "device": 'gpu',
                "num_classes": num_classes,
                "weights": False
            }
        },
        "trainer": {
            "default_root_dir": root_dir,
            "gpus": 1,
            "resume_from_checkpoint": resume_weights
        }
    }
}

uri = f'{RACEURL}/raceai/framework/inference'

resdata = json.loads(requests.post(url=uri, json=reqdata).text)
if 0 != resdata['errno']:
    if 'traceback' in resdata['result']:
        print(resdata['result']['traceback'])
    else:
        print(resdata)
else:
    print(resdata)

{'errno': 0, 'result': [{'image_id': '-1', 'image_path': '/tmp/tmpx_pc6cmk/39c1e05e0371cde9.jpg', 'probs': [0.0023420988582074642, 0.9976153373718262, 4.260418063495308e-05], 'probs_sorted': {'indices': [1, 0, 2], 'values': [0.9976153373718262, 0.0023420988582074642, 4.260418063495308e-05]}}, {'image_id': '-1', 'image_path': '/tmp/tmpx_pc6cmk/e621772e901d6d2f0ff072582e430173.jpg%40f_webp%2Cq_90', 'probs': [0.9999988079071045, 1.2288410289329477e-06, 4.06742373115776e-08], 'probs_sorted': {'indices': [0, 1, 2], 'values': [0.9999988079071045, 1.2288410289329477e-06, 4.06742373115776e-08]}}, {'image_id': '-1', 'image_path': '/tmp/tmpx_pc6cmk/2976462bd67042d5b1b14c1d0f4a6717.jpeg', 'probs': [5.383871126696249e-09, 3.1869282679508615e-07, 0.9999996423721313], 'probs_sorted': {'indices': [2, 1, 0], 'values': [0.9999996423721313, 3.1869282679508615e-07, 5.383871126696249e-09]}}, {'image_id': '-1', 'image_path': '/raceai/data/datasets/cleaner_robot/imgs/ring/ring_5.jpg', 'probs': [0.9998032450